# Examen Final



> Estudiante: Santillan Jason


> Dataset: [Genre Classification Dataset](https://www.kaggle.com/datasets/hijest/genre-classification-dataset-imdb/data) - Clasificación de Géneros de Peliculas










In [1]:
!pip install transformers==3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.6/754.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 14.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 78.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


## Importamos las librerías que se usarán

In [2]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from textwrap import wrap

# --------------------------------------------------------------------------------------------------------

In [3]:
from google.colab import drive
drive.mount('/content/drive')

# --------------------------------------------------------------------------------------------------------



Mounted at /content/drive


## Dataset

In [38]:
# Inicialización
RANDOM_SEED = 42
MAX_LEN = 200
BATCH_SIZE = 32
DATASET_PATH = '/content/drive/MyDrive/Datasets/Final/Genre Classification Dataset/train_data.txt'
NCLASSES = 27

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# --------------------------------------------------------------------------------------------------------



cuda:0


In [39]:
# Lectura del archivo y separación por triple dos puntos (:::)
# ID ::: TITLE ::: GENRE ::: DESCRIPTION

with open(DATASET_PATH, 'r', encoding='utf-8') as file:
    lines = file.readlines()

data = [line.strip().split(':::') for line in lines]

# Creación de la matriz de dos dimensiones con descripciones y géneros
descriptions = [line[3].strip() for line in data]
genres = [line[2].strip() for line in data]

# Creación del DataFrame con las columnas DESCRIPTION y GENRE
df = pd.DataFrame({'DESCRIPTION': descriptions, 'GENRE': genres})


# --------------------------------------------------------------------------------------------------------



In [40]:
# Imprimir las primeras filas y forma del nuevo dataset
print(df.head(), "\n")
print(df.shape, "\n")

# Revisar cómo lucen algunas descripciones y géneros
print("\n".join(wrap(df['DESCRIPTION'][200])), "\n")
print(df['GENRE'].unique())

# --------------------------------------------------------------------------------------------------------



                                         DESCRIPTION     GENRE
0  Listening in to a conversation between his doc...     drama
1  A brother and sister with a past incestuous re...  thriller
2  As the bus empties the students for their fiel...     adult
3  To help their unemployed father make ends meet...     drama
4  The film's title refers not only to the un-rec...     drama 

(54214, 2) 

Gun, A low-end, but knowledgeable gunrunner, and his temperamental
partner, Bully, earn a living selling firearms in shady motels.
Throughout their day, they encounter all sorts of unusual clientele,
which includes a serial killer, undercover cops and many more. The
story follows the duo on their final gun run, as they dash all over
the city, checking off their list. 

['drama' 'thriller' 'adult' 'documentary' 'comedy' 'crime' 'reality-tv'
 'horror' 'sport' 'animation' 'action' 'fantasy' 'short' 'sci-fi' 'music'
 'adventure' 'talk-show' 'western' 'family' 'mystery' 'history' 'news'
 'biography' 'roma

In [8]:
unique_categories = df['GENRE'].nunique()
print("Número de categorías únicas:", unique_categories)

Número de categorías únicas: 27


In [41]:
# Diccionario que mapea cada categoría a un número específico
category_mapping = {
    'drama': 0,
    'thriller': 1,
    'adult': 2,
    'documentary': 3,
    'comedy': 4,
    'crime': 5,
    'reality-tv': 6,
    'horror': 7,
    'sport': 8,
    'animation': 9,
    'action': 10,
    'fantasy': 11,
    'short': 12,
    'sci-fi': 13,
    'music': 14,
    'adventure': 15,
    'talk-show': 16,
    'western': 17,
    'family': 18,
    'mystery': 19,
    'history': 20,
    'news': 21,
    'biography': 22,
    'romance': 23,
    'game-show': 24,
    'musical': 25,
    'war': 26
}

# Reemplazar las categorías por los números correspondientes en la columna 'GENRE'
df['label'] = df['GENRE'].map(category_mapping)
# df.rename(columns={'DESCRIPTION': 'review'}, inplace=True)

# Eliminar la columna 'GENRE'
df.drop('GENRE', axis=1, inplace=True)

In [42]:
df.head()

,DESCRIPTION,label
0,Listening in to a conversation between his doc...,0
1,A brother and sister with a past incestuous re...,1
2,As the bus empties the students for their fiel...,2
3,To help their unemployed father make ends meet...,0
4,The film's title refers not only to the un-rec...,0


In [11]:
# TOKENIZACIÓN
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

# --------------------------------------------------------------------------------------------------------



tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [43]:
# Ejemplo tokenización
sample_txt = 'A committee investigating adults'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
# print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

# --------------------------------------------------------------------------------------------------------



Tokens:  ['A', 'committee', 'investigating', 'adults']
Tokens numéricos:  [138, 3914, 11950, 6323]


In [13]:
# Codificación para introducir a BERT
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length = 200,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    pad_to_max_length = True,
    return_attention_mask = True,
    return_tensors = 'pt'
)

# --------------------------------------------------------------------------------------------------------



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [14]:
encoding.keys()

# --------------------------------------------------------------------------------------------------------



dict_keys(['input_ids', 'attention_mask'])

In [15]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

# --------------------------------------------------------------------------------------------------------



['[CLS]', 'A', 'committee', 'investigating', 'adults', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
tensor([  101,   138,  3914, 11950,  6323,   102,     0,     0,     0,     0])
tensor([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])


In [16]:
# CREACIÓN DATASET

class IMDBDataset(Dataset):

  def __init__(self,reviews,labels,tokenizer,max_len):
    self.reviews = reviews
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
      return len(self.reviews)

  def __getitem__(self, item):
    review = str(self.reviews[item])
    label = self.labels[item]
    encoding = tokenizer.encode_plus(
        review,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        pad_to_max_length = True,
        return_attention_mask = True,
        return_tensors = 'pt'
        )


    return {
          'review': review,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'label': torch.tensor(label, dtype=torch.long)
      }


# --------------------------------------------------------------------------------------------------------



In [17]:
# Data loader:

def data_loader(df, tokenizer, max_len, batch_size):
  dataset = IMDBDataset(
      reviews = df.review.to_numpy(),
      labels = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )

  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 10)

# --------------------------------------------------------------------------------------------------------



In [18]:
df_train, df_test = train_test_split(df, test_size = 0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

# --------------------------------------------------------------------------------------------------------



/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
# Iterar sobre el DataLoader y mostrar el primer lote de datos
for batch in train_data_loader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    label = batch['label']

    print("input_ids shape:", input_ids.shape)
    print("attention_mask shape:", attention_mask.shape)
    print("label shape:", label.shape)

    # Imprimir una muestra de los datos en el primer lote
    print("input_ids:", input_ids[0])
    print("attention_mask:", attention_mask[0])
    print("label:", label[0])

    break  # Detener la iteración después del primer lote


input_ids shape: torch.Size([32, 200])
attention_mask shape: torch.Size([32, 200])
label shape: torch.Size([32])
input_ids: tensor([  101,  9850,   119, 26615,  6447,  1348,   119, 22087, 11243,   119,
         2268,  9717,  6856,   119, 24930, 15906,   119,  2185,   119, 20735,
          119,  3605,  1785,   119,   152,  4832,  5800,  1988,   119, 13212,
         1115,  4097,  1296,  1104,  1103,   129,  2650,  1104, 17730, 14569,
         2036,   119,  8138,   117,  5331,   117, 14096,   117, 25662,   117,
        17500,  3285,   117,   141,   118,  8274,   117, 17979,  1105,  1287,
          119,  2994,   170,  3283,  1120, 11124,  4727, 19187, 20122,  1116,
         1105,  1296,  5898,  1916, 15801,   176,  2312,  9208,  1116,  1104,
         1260, 20488, 14499,   119, 17730, 14569,  2036, 14715, 25981,  1116,
         6765,   117,  1260, 19248,  1818,  1105,  1159,   117,   185, 24356,
         1158,  1103, 15403,  1205,  1106,  1103,   170, 24598,  1107,  1134,
         1195,  11

In [20]:
# EL MODELO!

class BERTSentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BERTSentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    outputs = self.bert(
                        input_ids = input_ids,
                        attention_mask = attention_mask
                        )
    pooled_output = outputs.pooler_output
    drop_output = self.drop(pooled_output)
    output = self.linear(drop_output)
    return output


# --------------------------------------------------------------------------------------------------------



In [21]:
model = BERTSentimentClassifier(NCLASSES)
model = model.to(device)

# --------------------------------------------------------------------------------------------------------



model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [22]:
print(model)


BERTSentimentClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [23]:
# ENTRENAMIENTO
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device)

# --------------------------------------------------------------------------------------------------------



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [24]:

# Iteración entrenamiento
def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    total_batches = len(data_loader)
    batches_to_check = total_batches // 10  # Determinar cada cuántos lotes se mostrará el progreso
    for batch_idx, batch in enumerate(data_loader, 1):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Mostrar progreso cada 10% de los batches
        if batch_idx % batches_to_check == 0 or batch_idx == total_batches:
            acc = correct_predictions.double() / (batch_idx * data_loader.batch_size)
            avg_loss = np.mean(losses)
            print(f'Batch [{batch_idx}/{total_batches}], Accuracy: {acc:.4f}, Loss: {avg_loss:.4f}')

    return correct_predictions.double() / n_examples, np.mean(losses)


def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids = input_ids, attention_mask = attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)


# --------------------------------------------------------------------------------------------------------



In [25]:
# Entrenamiento!!!

for epoch in range(EPOCHS):
  print('Epoch {} de {}'.format(epoch+1, EPOCHS))
  print('------------------')
  train_acc, train_loss = train_model(
      model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
  )
  test_acc, test_loss = eval_model(
      model, test_data_loader, loss_fn, device, len(df_test)
  )
  print('Entrenamiento: Loss: {}, accuracy: {}'.format(train_loss, train_acc))
  print('Validación: Loss: {}, accuracy: {}'.format(test_loss, test_acc))
  print('')

# --------------------------------------------------------------------------------------------------------



Epoch 1 de 5
------------------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.wa

Batch [135/1356], Accuracy: 0.4225, Loss: 2.0233
Batch [270/1356], Accuracy: 0.4902, Loss: 1.7995
Batch [405/1356], Accuracy: 0.5248, Loss: 1.6882
Batch [540/1356], Accuracy: 0.5432, Loss: 1.6117
Batch [675/1356], Accuracy: 0.5590, Loss: 1.5449
Batch [810/1356], Accuracy: 0.5696, Loss: 1.5008
Batch [945/1356], Accuracy: 0.5787, Loss: 1.4617
Batch [1080/1356], Accuracy: 0.5863, Loss: 1.4287
Batch [1215/1356], Accuracy: 0.5923, Loss: 1.4037
Batch [1350/1356], Accuracy: 0.5964, Loss: 1.3839
Batch [1356/1356], Accuracy: 0.5961, Loss: 1.3834


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

Entrenamiento: Loss: 1.3833583576884945, accuracy: 0.5964354061469646
Validación: Loss: 1.1503390360370849, accuracy: 0.6544314304159365

Epoch 2 de 5
------------------


...

Entrenamiento: Loss: 0.5097359855519196, accuracy: 0.8582232367250006
Validación: Loss: 1.3454703504884489, accuracy: 0.6583048971686802

Epoch 5 de 5
------------------


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

Batch [135/1356], Accuracy: 0.8725, Loss: 0.4514
Batch [270/1356], Accuracy: 0.8829, Loss: 0.4136
Batch [405/1356], Accuracy: 0.8878, Loss: 0.4089
Batch [540/1356], Accuracy: 0.8887, Loss: 0.4055
Batch [675/1356], Accuracy: 0.8907, Loss: 0.3977
Batch [810/1356], Accuracy: 0.8897, Loss: 0.4027
Batch [945/1356], Accuracy: 0.8904, Loss: 0.3990
Batch [1080/1356], Accuracy: 0.8892, Loss: 0.4018
Batch [1215/1356], Accuracy: 0.8878, Loss: 0.4089
Batch [1350/1356], Accuracy: 0.8859, Loss: 0.4151
Batch [1356/1356], Accuracy: 0.8854, Loss: 0.4155


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input 

Entrenamiento: Loss: 0.41552961181442644, accuracy: 0.8857992667911738
Validación: Loss: 1.4205544108310633, accuracy: 0.6525869224384395



In [26]:
torch.save(model.state_dict(), "bert_classifier.pth")

In [30]:
def predict_genre(text, model, tokenizer, device, max_length=200):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, predicted_class = torch.max(outputs, dim=1)

    # Mapear el número de clase predicho al nombre de la categoría correspondiente
    predicted_genre = list(category_mapping.keys())[list(category_mapping.values()).index(predicted_class.item())]
    return predicted_genre


In [35]:
test_text = "A young artist boards the famous Titanic on its maiden voyage. There, he falls in love with a young woman from high society engaged to another man. As the ship sails towards its destiny, an inevitable tragedy changes their lives forever."
genre = predict_genre(test_text, model, tokenizer, device)
print("\n".join(wrap(test_text)))
print(f"\nPredicted genre: {genre}")

A young artist boards the famous Titanic on its maiden voyage. There,
he falls in love with a young woman from high society engaged to
another man. As the ship sails towards its destiny, an inevitable
tragedy changes their lives forever.

Predicted genre: drama


In [36]:
test_text = "In a galaxy far, far away, a young farm boy embarks on an epic journey after discovering a message hidden within a droid. He teams up with a wise old Jedi Knight, a smuggler, and their loyal companions to rescue a princess and face the sinister forces of the Empire."
genre = predict_genre(test_text, model, tokenizer, device)
print("\n".join(wrap(test_text)))
print(f"\nPredicted genre: {genre}")


In a galaxy far, far away, a young farm boy embarks on an epic journey
after discovering a message hidden within a droid. He teams up with a
wise old Jedi Knight, a smuggler, and their loyal companions to rescue
a princess and face the sinister forces of the Empire.

Predicted genre: sci-fi
